In [1]:
import onnx
import tvm
from tvm import relay
from tvm.contrib import graph_executor
from tvm import autotvm
from tvm.autotvm.tuner import XGBTuner
import numpy as np

In [2]:
onnx_model_path = "model.onnx"  
onnx_model = onnx.load(onnx_model_path)

In [3]:
target = "llvm" 
log_file = "tuning.log"  

In [4]:
input_name = "input"  
input_shape = (1, 28, 28, 1)

In [5]:
shape_dict = {input_name: input_shape}

mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

In [6]:
tasks = autotvm.task.extract_from_program(mod["main"], target=target, params=params)

tuning_option = {
    "log_filename": log_file,
    "tuner": "xgb", 
    "trials": 1000,  
    "early_stopping": 100, 
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(), runner=autotvm.LocalRunner(repeat=3, timeout=10)
    ),
}

In [ ]:
for i, task in enumerate(tasks):
    print(f"Tuning task {i+1}/{len(tasks)}: {task.name}")
    tuner = XGBTuner(task)
    tuner.tune(
        n_trial=tuning_option["trials"],
        early_stopping=tuning_option["early_stopping"],
        measure_option=tuning_option["measure_option"],
        callbacks=[autotvm.callback.log_to_file(tuning_option["log_filename"])],
    )
print("Tuning complete. Log saved to:", log_file)

In [8]:
with autotvm.apply_history_best(log_file):
    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod, target=target, params=params)

In [9]:
dev = tvm.cpu(0)  
module = graph_executor.GraphModule(lib["default"](dev))

In [10]:
dummy_input = np.random.rand(*input_shape).astype("float32")
module.set_input(input_name, dummy_input)
module.run()

In [ ]:
lib.export_library("optimized_tvm_model_with_tuning.so")
print("Optimized TVM model saved as 'optimized_tvm_model_with_tuning.so'")